<a href="https://colab.research.google.com/github/dajebbar/FreeCodeCamp-python-data-analysis/blob/main/num_cat_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Using numerical and categorical variables together
---


In [1]:
import pandas as pd

adult_census = pd.read_csv("./adult.csv")
# drop the duplicated column `"education-num"` as stated in the first notebook
adult_census = adult_census.drop(columns="education-num")

target_name = "class"
target = adult_census[target_name]

data = adult_census.drop(columns=[target_name])

## Selection based on data types
We will separate categorical and numerical variables using their data types to identify them, as we saw previously that object corresponds to categorical columns (strings). We make use of `make_column_selector` helper to select the corresponding columns.



In [8]:
from sklearn.compose import make_column_selector as selector

numerical_data_selector = selector(dtype_include='number')
categorical_data_selector = selector(dtype_include='object')

numerical_columns = numerical_data_selector(data)
categorical_columns = categorical_data_selector(data)

In [9]:
categorical_columns

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

## Dispatch columns to a specific processor

In the previous sections, we saw that we need to treat data differently depending on their nature (i.e. numerical or categorical).

Scikit-learn provides a `ColumnTransformer` class which will send specific columns to a specific transformer, making it easy to fit a single predictive model on a dataset that combines both kinds of variables together (heterogeneously typed tabular data).

We first define the columns depending on their data type:

-  **one-hot encoding** will be applied to categorical columns. Besides, we use `handle_unknown="ignore"` to solve the potential issues due to rare categories.
-  **numerical scaling** numerical features which will be standardized.  

Now, we create our `ColumnTransfomer` by specifying three values: the preprocessor name, the transformer, and the columns. First, let's create the preprocessors for the numerical and categorical parts.

In [10]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_preprocessor = OneHotEncoder(handle_unknown='ignore')
numerical_preprocessor = StandardScaler()

Now, we create the transformer and associate each of these preprocessors with their respective columns.

In [11]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer([
                                  ('one-hot-encoder', 
                                   categorical_preprocessor, 
                                   categorical_columns),
                                  ('standard-scaler', 
                                  numerical_preprocessor, 
                                   numerical_columns),
])

A `ColumnTransformer` does the following:

-  It **splits the columns** of the original dataset based on the column names or indices provided. We will obtain as many subsets as the number of transformers passed into the `ColumnTransformer`.
-  It **transforms each subsets**. A specific transformer is applied to each subset: it will internally call `fit_transform` or `transform`. The output of this step is a set of transformed datasets.
-  It then **concatenates the transformed datasets** into a single dataset.  

The important thing is that `ColumnTransformer` is like any other scikit-learn transformer. In particular it can be combined with a classifier in a `Pipeline`:

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))

We can display an interactive diagram with the following command:

In [13]:
from sklearn import set_config
set_config(display='diagram')
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country']),
                                                 ('standard-scaler',
                                                  StandardScaler(),
                                                  ['age', 'fnlwgt',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

The final model is more complex than the previous models but still follows the same API (the same set of methods that can be called by the user):

-  the `fit` method is called to preprocess the data and then train the classifier of the preprocessed data;
-  the `predict` method makes predictions on new data;
-  the `score` method is used to predict on the test data and compare the predictions to the expected test labels to compute the accuracy.  

Let's start by splitting our data into train and test sets.

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data,
    target,
    random_state=42
)

Now, we can train the model on the train set.

In [15]:
_= model.fit(X_train, y_train)

Then, we can send the raw dataset straight to the pipeline. Indeed, we do not need to make any manual preprocessing (calling the `transform` or `fit_transform` methods) as it will be handled when calling the `predict` method. As an example, we predict on the five first samples from the test set.

In [16]:
X_test.head()

,age,workclass,fnlwgt,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
7762,56,Private,33115,HS-grad,Divorced,Other-service,Unmarried,White,Female,0,0,40,United-States
23881,25,Private,112847,HS-grad,Married-civ-spouse,Transport-moving,Own-child,Other,Male,0,0,40,United-States
30507,43,Private,170525,Bachelors,Divorced,Prof-specialty,Not-in-family,White,Female,14344,0,40,United-States
28911,32,Private,186788,HS-grad,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States
19484,39,Private,277886,Bachelors,Married-civ-spouse,Sales,Wife,White,Female,0,0,30,United-States


In [17]:
model.predict(X_test)[:5]

array([' <=50K', ' <=50K', ' >50K', ' <=50K', ' >50K'], dtype=object)

In [18]:
y_test[:5]

7762      <=50K
23881     <=50K
30507      >50K
28911     <=50K
19484     <=50K
Name: class, dtype: object

To get directly the accuracy score, we need to call the score method. Let's compute the accuracy score on the entire test set.

In [20]:
model.score(X_test, y_test)

0.8583244615510606

## Evaluation of the model with cross-validation
As previously stated, a predictive model should be evaluated by cross-validation. Our model is usable with the cross-validation tools of scikit-learn as any other predictors:

In [21]:
from sklearn.model_selection import cross_validate, KFold

kfold = KFold(n_splits=10, shuffle=True, random_state=42)
cv_results = cross_validate(model,
                            data,
                            target,
                            cv=kfold)

cv_results

{'fit_time': array([3.7650156 , 2.37976193, 2.32669282, 2.2351191 , 2.22663641,
        2.30055547, 2.45320868, 2.32616472, 2.71137977, 2.29445505]),
 'score_time': array([0.02877688, 0.02840304, 0.02745271, 0.02725101, 0.02720189,
        0.02994108, 0.02924705, 0.03813934, 0.0273807 , 0.02701426]),
 'test_score': array([0.85772774, 0.85609007, 0.86036036, 0.83947584, 0.85298935,
        0.85421785, 0.85176085, 0.85257985, 0.8503276 , 0.84766585])}

In [22]:
scores = cv_results["test_score"]
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

The mean cross-validation accuracy is: 0.852 +/- 0.006


The compound model has a higher predictive accuracy than the two models that used numerical and categorical variables in isolation.

---
## Fitting a more powerful model
**Linear models** are nice because they are usually cheap to train, **small** to deploy, **fast** to predict and give a **good baseline**.

However, it is often useful to check whether more complex models such as an ensemble of decision trees can lead to higher predictive performance. In this section we will use such a model called **gradient-boosting trees** and evaluate its generalization performance. More precisely, the scikit-learn model we will use is called `HistGradientBoostingClassifier`. Note that boosting models will be covered in more detail in a future module.

For tree-based models, the handling of numerical and categorical variables is simpler than for linear models:

-  we do **not need to scale the numerical features**
-  using an **ordinal encoding for the categorical variables** is fine even if the encoding results in an arbitrary ordering  

Therefore, for `HistGradientBoostingClassifier`, the preprocessing pipeline is slightly simpler than the one we saw earlier for the `LogisticRegression`: